In [1]:
import os

from summarizer.sbert import SBertSummarizer
import torch

from tqdm.notebook import tqdm

from db.DocumentRepository import DocumentRepository

/Users/danielwps/Documents/Private/Github/Project_MSE/venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
os.system("""
    docker compose down;
    docker compose up -d --build db;
    sleep 3;
    """)

In [2]:
documentRepository = DocumentRepository()
tokenizer = documentRepository.tokenizer

SC: Connected to the db. Now you can go and build the best search engine around!


In [3]:
summarizer = SBertSummarizer('paraphrase-MiniLM-L6-v2')
summarizer

In [18]:
all_docs = documentRepository.loadAllDocuments()
len(all_docs)

6355

In [19]:
print(all_docs[0].enc_text)
print(all_docs[0].summary)

tensor([ 6160.,  1013.,  4895.,  5480., 22153.,  2243., 14366., 15542., 11265.,
        10976.,  5428.,  6820., 10623.,  2666.,  7632.,  2078., 19845.,  2015.,
          999.,  8289.,  2063.,  2019., 12449., 27584.,  2290.,  3841.,  4140.,
         8004.,  2102.,  9262.,  5896.,  1012.,  4315.,  4371.,  4183.,  5292.,
        10609.,  9033.,  2063.,  9262.,  5896.,  1999.,  1045., 28362.,  2213.,
        16602., 26709., 22462., 14356.,  2102.,  1012., 16417.,  2019., 23057.,
        21847., 15536.,  2063.,  9033.,  2063.,  9262.,  5896.,  1999.,  1045.,
        28362.,  2213., 16602., 17712., 29068., 21253.,  2078.,  2424.,  2368.,
         9033.,  2063.,  7632.,  2121.,  1012., 11265., 10976.,  5428.,  6820.,
        10623.,  2666., 14366., 15542., 17894.,  2819.,  2139., 16446.,  2818.,
         2394.,  3945.,  4037.,  2097.,  3683.,  3549., 21388.,  4315., 11265.,
        10976.,  5428.,  6820., 10623.,  2666.,  4315.,  4895.,  5480., 22153.,
         2243., 14366., 15542.,  2533., 

In [14]:
print(len(all_docs))
number_none_enc_text = [doc.enc_text for doc in all_docs].count(None)
number_none_summary = [doc.summary for doc in all_docs].count(None)
print(f"The database should not contain any document without the encoded page text and summary. Is test passed: {number_none_enc_text == 0 and number_none_summary == 0}")

6355
The database should not contain any document without the encoded page text and summary. Is test passed: True


If the test is giving you "False", you should go and execute the other cells as well. If you got "True", you are done with this notebook already. :)

In [ ]:
# 1. Step: Encode the text
for doc in tqdm(all_docs):
    text = doc.page_text
    
    encode = tokenizer.encode(text, add_special_tokens=False)
    doc_vec = torch.tensor(encode)
    doc.enc_text = doc_vec

In [13]:
# 2. Step: Summarize the text
for doc in tqdm(all_docs):
    text = doc.page_text
    
    summary = summarizer(text, min_length=60, num_sentences=3, max_length=300)
    doc.summary = summary

  0%|          | 0/6355 [00:00<?, ?it/s]

In [ ]:
print(len(new_all_docs))
number_none_enc_text = [doc.enc_text for doc in new_all_docs].count(None)
print(f"The database should not contain any document without the encoded page text anymore. Is test passed: {number_none_enc_text == 0}")

In [15]:
for doc in tqdm(all_docs):
    documentRepository.updateDocument(doc)

  0%|          | 0/6355 [00:00<?, ?it/s]

SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Update

In [16]:
new_all_docs = documentRepository.loadAllDocuments()
print(len(all_docs))
number_none_enc_text = [doc.enc_text for doc in new_all_docs].count(None)
number_none_summary = [doc.summary for doc in new_all_docs].count(None)
print(f"The database should not contain any document without the encoded page text and summary. Is test passed: {number_none_enc_text == 0 and number_none_summary == 0}")

6355
The database should not contain any document without the encoded page text and summary. Is test passed: True


In [17]:
if number_none_enc_text == 0 and number_none_summary == 0:
    documentRepository.overwrite_dump()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


SC: Successfully overwritten the old dump. Now you only need to push it to the repository!


Our current index is too big for ranking BM25 with all of them during inference. Let's shorten it

In [20]:
cutted_docs = new_all_docs[:200]
documentRepository.deleteAllDocuments()
documentRepository.saveAllDocuments(cutted_docs)
saved_docs = documentRepository.loadAllDocuments()
len(saved_docs)

SC: Deleted all documents.
SC: All documents saved.


200